In [1]:
import torch
import numpy as np
import torch.nn as nn
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader, TensorDataset
import src.generate_encodings as ge
import math
import random
import datetime
import time
from tqdm import tqdm
from sklearn.metrics import r2_score
import warnings

warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'src.generate_encodings'

In [2]:
print(torch.cuda.is_available())
torch.cuda.get_device_name(0)

True


'NVIDIA GeForce RTX 4060 Ti'

In [3]:
"""
Load ESM Representations
Warning: Only use this block or the block below. Not both!
"""

torch.cuda.empty_cache()
random.seed(0)

data_set = "HIS7_YEAST_Pokusaeva_2019"
method = "esmc_600m"
representations = []
repr_layer = "-1"
scores = []
mapping_file = f"../Data/Embeddings/{data_set}/pairings.csv"
path_to_embeddings = f"../Data/Embeddings//HIS7_YEAST_Pokusaeva_2019/{method}/{repr_layer}"
with open(mapping_file) as f:
    data_pairs = [line[:-1] for line in f.readlines()[1:]]
    random.shuffle(data_pairs)

with tqdm(total=len(data_pairs)) as pbar:
    for line in data_pairs:
        line = line[:-1].split(',')
        embedding = f"{path_to_embeddings}/{line[0][1:]}"
        representations.append(torch.load(embedding).to(dtype=torch.float32).cpu())
        scores.append(float(line[1]))
        pbar.update(1)

print("successfully loaded all embeddings and their scores to cpu")

100%|██████████| 496137/496137 [04:53<00:00, 1691.99it/s]

successfully loaded all embeddings and their scores to cpu


In [ ]:
"""
Generate Sequence Encodings + OHE
Warning: Only use this block or the block above. Not both!
"""
data_set = "CAPSD_AAV2S_Sinai_2021"
data_file = f"../Data/Protein_Gym_Datasets/{data_set}.csv"

representations = []
scores = []
embedding_type = "georgiev"
with open(data_file) as f:
    data_pairs = [(line[:-1].split(",")[1][1:], line[:-1].split(",")[2]) for line in f.readlines()[1:]]

with tqdm(total=len(data_pairs)) as pbar:
    for sequence, score in data_pairs:
        sequence = sequence
        score = round(float(score), 3)
        emb1 = ge.generate_sequence_encodings(embedding_type, [sequence])[0].reshape(-1).tolist()
        ohe = ge.generate_sequence_encodings("one_hot", [sequence])[0].reshape(-1).tolist()
        for binary in ohe:
            emb1.append(binary)

        representations.append(torch.tensor(emb1, dtype=torch.float32).cpu())
        scores.append(score)
        pbar.update(1)

In [4]:
print(np.array(representations)[1])

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7efd64745ca0>>
Traceback (most recent call last):
  File "/opt/conda/conda/envs/MAP/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


[[ 1.2000000e+01 -2.3375000e+01  7.8437500e+00 -6.6250000e+00
   3.3750000e+01 -1.1000000e+01 -9.0625000e+00 -1.6500000e+01
  -2.0000000e+01  1.5937500e+01 -4.9062500e+00 -2.9875000e+01
   1.0750000e+01  4.8250000e+01 -2.3500000e+01 -1.9531250e+00
   3.8000000e+01 -4.4140625e-01 -2.5750000e+01  1.3812500e+01
   4.9750000e+01  3.7750000e+01 -8.0000000e+00  3.9600000e+02
  -5.3250000e+01 -3.3000000e+01 -2.3000000e+01  7.9500000e+01
  -1.4843750e+00  4.2500000e+01 -2.8750000e+01 -5.5859375e-01
   3.4750000e+01  1.6000000e+01  3.8750000e+01  6.8000000e+01
  -5.8750000e+01 -3.1054688e-01  1.3125000e+01 -1.7750000e+01
  -4.6000000e+01  2.4125000e+01  1.3937500e+01  1.9625000e+01
  -2.0000000e+01 -2.3500000e+01  1.2312500e+01  4.8750000e+00
  -5.1500000e+01  3.0875000e+01 -4.6750000e+01 -2.7750000e+01
  -7.7734375e-01  7.8500000e+01 -1.0312500e+01 -4.7187500e+00
  -1.1187500e+01 -2.3375000e+01 -1.0875000e+01 -1.7187500e-01
  -1.4609375e+00  3.1500000e+01 -3.5000000e+01 -7.9062500e+00
  -6.312

In [5]:
split_ratio = (0.75, 0.15, 0.10)
split_train = int(len(representations) * split_ratio[0])
split_valid = int(len(representations) * (split_ratio[1] + split_ratio[0]))

# if isinstance(representations[0], torch.Tensor):
x_train = torch.stack([x.reshape(-1).to(dtype=torch.float32) for x in representations[0:split_train]])
x_valid = torch.stack([x.reshape(-1).to(dtype=torch.float32) for x in representations[split_train:split_valid]])
x_test = torch.stack([x.reshape(-1).to(dtype=torch.float32) for x in representations[split_valid:]])

y_train = torch.stack([torch.tensor(y, dtype=torch.float32) for y in scores[0:split_train]])
y_valid = torch.stack([torch.tensor(y, dtype=torch.float32) for y in scores[split_train: split_valid]])
y_test = torch.stack([torch.tensor(y, dtype=torch.float32) for y in scores[split_valid:]])

train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=1000, shuffle=False)

test_data = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_data, batch_size=1000, shuffle=False)

valid_data = TensorDataset(x_valid, y_valid)
valid_loader = DataLoader(valid_data, batch_size=1000, shuffle=False)

input_len = len(x_train[0])
print(input_len)


960


In [ ]:
import gc
del representations
del scores
gc.collect()
torch.cuda.empty_cache()

In [7]:
class FFNet(nn.Module):
    def __init__(self):
        super(FFNet, self).__init__()

        hidden_size = int(input_len / 4)

        self.fc1 = nn.Linear(input_len, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        # self.fc4 = nn.Linear(hidden_size, hidden_size)
        # self.fc5 = nn.Linear(hidden_size, hidden_size)
        # self.fc6 = nn.Linear(hidden_size, hidden_size)
        # self.fc7 = nn.Linear(hidden_size, hidden_size)
        # self.fc8 = nn.Linear(hidden_size, hidden_size)
        # self.fc9 = nn.Linear(hidden_size, hidden_size)
        # self.fc10 = nn.Linear(hidden_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, 1)
        # leaky = nn.LeakyReLU(0.1)
        # self.output_layer = leaky(self.output_layer(input_len, 1))
        # self.output_layer = nn.Linear(input_len, 1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        # x = torch.relu(self.fc4(x))
        # x = torch.relu(self.fc5(x))
        # x = torch.relu(self.fc6(x))
        # x = torch.relu(self.fc7(x))
        # x = torch.relu(self.fc8(x))
        # x = torch.relu(self.fc9(x))
        # x = torch.relu(self.fc10(x))
        x = self.output_layer(x)
        return x

# x = torch.relu(self.fc4(x))
# x = torch.relu(self.fc5(x))
# x = torch.relu(self.fc6(x))
# x = torch.relu(self.fc7(x))
# x = torch.relu(self.fc8(x))
# x = torch.relu(self.fc9(x))
# x = torch.relu(self.fc10(x))
# x = torch.relu(self.fc11(x))
# x = torch.relu(self.fc12(x))
# x = torch.relu(self.fc13(x))
# x = torch.relu(self.fc14(x))
# x = torch.relu(self.fc15(x))
# x = torch.relu(self.fc16(x))
# x = torch.relu(self.fc17(x))
# x = torch.relu(self.fc18(x))
# x = torch.relu(self.fc19(x))
# x = torch.relu(self.fc20(x))



In [17]:
"""Creating up the neural net + log"""
gt_model = FFNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gt_model.to(device, dtype=torch.float32)
criterion = nn.MSELoss()

learning_rate = 0.000003
momentum = 0.1
optim = "ADAM"
if optim == "SGD":
    optimizer = torch.optim.SGD(gt_model.parameters(), lr=learning_rate, momentum=momentum)  #, momentum=momentum)
else:
    optimizer = torch.optim.Adam(gt_model.parameters(), lr=learning_rate / 100)

In [18]:
data_set = "HIS7_YEAST_Pokusaeva_2019"
embedding_type = "esmc_300m"







"""Track Training Metrics"""
time_stamp = datetime.datetime.now().strftime("%Y_%m_%d-%H_%M")
outfile = f"../Models/Logs/{time_stamp}.txt"
with open(outfile, "w") as f:
    f.write(
        f"Dataset: {data_set}, Encoding-Method: {embedding_type}, Optimizer: {optim}, criterion: {criterion},learning_rate: {learning_rate}, momentum: {momentum} \n")
    f.write(f"Model: \n {gt_model} \n")
    f.write("\n")

In [19]:
torch.cuda.empty_cache()
n_epochs = 500
diplay_every_n_epochs = 1

for epoch in range(n_epochs):
    start_time = time.time()
    gt_model.train()
    running_loss = 0.0
    tr_y_pred = []
    tr_y_true = []

    best_val = 1
    best_model = None
    optimizer.zero_grad()

    for tr_inputs, tr_labels in train_loader:
        # Move data to the same device as the model
        tr_inputs = tr_inputs.to(device)
        tr_labels = tr_labels.to(device)

        # optimizer.zero_grad()
        tr_outputs = gt_model(tr_inputs)  # Forward pass

        loss = criterion(tr_outputs, tr_labels)
        running_loss += loss.item()
        for i, result in enumerate(tr_outputs):
            result_ = result.cpu().detach().numpy().tolist()[0]
            tr_y_pred.append(result_)
        for i, label in enumerate(tr_labels):
            label_ = label.cpu().detach().numpy().tolist()
            tr_y_true.append(label_)
        loss.backward()
        # optimizer.step()
    optimizer.step()
    r2_train = round(r2_score(tr_y_true, tr_y_pred),3)
    # scheduler.step()

    gt_model.eval()
    with torch.no_grad():
        te_loss = 0.0
        running_va_loss = 0.0

        va_y_pred = []
        va_y_true = []

        for va_input, va_label in valid_loader:
            # Move data to the same device as the model
            va_input = va_input.to(device)
            va_labels = va_label.to(device)
            va_outputs = gt_model(va_input) # Forward pass
            va_loss = criterion(va_outputs.cpu(), va_label.cpu())
            running_va_loss += va_loss.item()
            for i, result in enumerate(va_outputs):
                result_ = result.cpu().detach().numpy().tolist()[0]
                va_y_pred.append(result_)
            for i, label in enumerate(va_labels):
                label_ = label.cpu().detach().numpy().tolist()
                va_y_true.append(label_)


        r2_val = round(r2_score(va_y_true, va_y_pred),3)
        tr_RMSE = math.sqrt(running_loss / len(train_loader))
        va_RMSE = math.sqrt(running_va_loss / len(valid_loader))

    with open(outfile, "a") as f:
        f.write(
            f'[Epoch {epoch + 1}/{n_epochs}, t: {round(time.time() - start_time, 2)}s]:tLoss: {running_loss / len(train_loader):.4f}, tR2: {r2_train}, tRMSE: {tr_RMSE:.4f}, '
            f'vLoss: {running_va_loss / len(valid_loader):.4f}, vR2: {r2_val}, vRMSE: {va_RMSE:.4f} \n')  #,

    if (epoch + 1) % diplay_every_n_epochs == 0:
        print(
            f'[Epoch {epoch + 1}/{n_epochs}, t: {round(time.time() - start_time, 2)}s]:tLoss: {running_loss / len(train_loader):.4f}, tR2: {r2_train}, tRMSE: {tr_RMSE:.4f}, '
            f'vLoss: {running_va_loss / len(valid_loader):.4f}, vR2: {r2_val}, vRMSE: {va_RMSE:.4f}')


[Epoch 1/500, t: 9.28s]:tLoss: 0.5962, tR2: -2.057, tRMSE: 0.7722, vLoss: 0.5978, vR2: -2.062, vRMSE: 0.7732
[Epoch 2/500, t: 9.44s]:tLoss: 0.5944, tR2: -2.047, tRMSE: 0.7710, vLoss: 0.5960, vR2: -2.053, vRMSE: 0.7720
[Epoch 3/500, t: 9.57s]:tLoss: 0.5926, tR2: -2.038, tRMSE: 0.7698, vLoss: 0.5941, vR2: -2.044, vRMSE: 0.7708
[Epoch 4/500, t: 9.65s]:tLoss: 0.5907, tR2: -2.029, tRMSE: 0.7686, vLoss: 0.5923, vR2: -2.035, vRMSE: 0.7696
[Epoch 5/500, t: 9.41s]:tLoss: 0.5889, tR2: -2.02, tRMSE: 0.7674, vLoss: 0.5905, vR2: -2.026, vRMSE: 0.7684
[Epoch 6/500, t: 9.35s]:tLoss: 0.5871, tR2: -2.011, tRMSE: 0.7662, vLoss: 0.5886, vR2: -2.016, vRMSE: 0.7672
[Epoch 7/500, t: 9.62s]:tLoss: 0.5853, tR2: -2.002, tRMSE: 0.7650, vLoss: 0.5868, vR2: -2.007, vRMSE: 0.7660
[Epoch 8/500, t: 9.31s]:tLoss: 0.5835, tR2: -1.993, tRMSE: 0.7638, vLoss: 0.5850, vR2: -1.998, vRMSE: 0.7648
[Epoch 9/500, t: 9.35s]:tLoss: 0.5816, tR2: -1.983, tRMSE: 0.7627, vLoss: 0.5832, vR2: -1.989, vRMSE: 0.7637
[Epoch 10/500, t: 9.

KeyboardInterrupt: 